# # Chord Progressions Notebook
Este notebook demuestra cómo usar nuestros módulos para:
1. Generar progresiones `.mid`
2. Convertirlas a `.wav` con Timidity
3. Crear anotaciones `.jams`

## 1. Imports y Configuración

In [1]:
import sys
import os
from pathlib import Path

# Asegurarnos de que Python vea nuestro paquete "src/"
module_path = Path.cwd().parent  # Ajustar si es necesario
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))

from src.generate_progression import generate_progression
from src.audio_conversion import convert_all_mid_in_folder
from src.jams_creation import create_jams_for_folder
from src.config import MIDI_DIR, WAV_DIR, JAMS_DIR

# Verificamos rutas
print("MIDI_DIR:", MIDI_DIR)
print("WAV_DIR:", WAV_DIR)
print("JAMS_DIR:", JAMS_DIR)

MIDI_DIR: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi
WAV_DIR: /home/cepatinog/MIR/final_project/chord_progression_generator/data/wav
JAMS_DIR: /home/cepatinog/MIR/final_project/chord_progression_generator/data/jams


## 2. Generar Progresiones Ejemplo
Probaremos dos progresiones de 3 acordes, y una de 4 acordes, por ejemplo:
- `ii-V-I` (3 acordes)
- `I-vi-IV-V` (4 acordes)


In [2]:
# Progresión 1: ii-V-I
progression_1 = "ii-V-I"
progression_name_1 = "ii-V-I_demo"
generate_progression(progression_1, progression_name_1)

# Progresión 2: I-vi-IV-V
progression_2 = "I-vi-IV-V"
progression_name_2 = "I-vi-IV-V_demo"
generate_progression(progression_2, progression_name_2)

print("¡Progresiones generadas en data/midi!")


Generated progression 'ii-V-I' -> folder: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo
Generated progression 'I-vi-IV-V' -> folder: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/I-vi-IV-V_demo
¡Progresiones generadas en data/midi!


## 3. Convertir los .mid a .wav usando Timidity
Convertimos todo lo que esté en `data/midi/ii-V-I_demo/` y `data/midi/I-vi-IV-V_demo/` hacia `data/wav`.


In [3]:
# Convertir "ii-V-I_demo"
midi_folder_1 = MIDI_DIR / progression_name_1
convert_all_mid_in_folder(midi_folder_1)

# Convertir "I-vi-IV-V_demo"
midi_folder_2 = MIDI_DIR / progression_name_2
convert_all_mid_in_folder(midi_folder_2)

print("Archivos WAV generados en data/wav!")


Playing /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo/D-6-ii-V-I_demo-22.mid
MIDI file: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo/D-6-ii-V-I_demo-22.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~10 seconds
Notes cut: 0
Notes lost totally: 0
Convertido: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo/D-6-ii-V-I_demo-22.mid => /home/cepatinog/MIR/final_project/chord_progression_generator/data/wav/D-6-ii-V-I_demo-22.wav
Playing /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo/Bb-7-ii-V-I_demo-17.mid
MIDI file: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo/Bb-7-ii-V-I_demo-17.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~10 seconds
Notes cut: 0
Notes lost totally: 0
Convertido: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii-V-I_demo/Bb-7-ii-V-I_demo-17.mid 

Terminated sig=0x02


KeyboardInterrupt: 

## 4. Crear archivos .jams con etiquetas de acordes
Supondremos que cada acorde dura 2 segundos en la reproducción. 
Etiquetamos cada acorde con notación mixta:
- Por ejemplo: "ii (Dmin7)", "V (G7)", "I (Cmaj7)" si asumimos tonalidad de C
o
- "I (C)", "vi (Am)", "IV (F)", "V (G)" si es C mayor


In [4]:
# Para la progresión ii-V-I asumimos tonalidad de C, sin tensiones
# NOTA: en otras tonalidades cambia!
chord_seq_1 = ["ii (Dmin7)", "V (G7)", "I (Cmaj7)"]
create_jams_for_folder(midi_folder_1, chord_seq_1, progression_name_1)

# Para la progresión I-vi-IV-V en C mayor
chord_seq_2 = ["I (C)", "vi (Am)", "IV (F)", "V (G)"]
create_jams_for_folder(midi_folder_2, chord_seq_2, progression_name_2)

print("Archivos .jams generados en data/jams!")


SchemaError: 'ii (Dmin7)' does not match '^((N|X)|(([A-G](b*|#*))((:(maj|min|dim|aug|1|5|sus2|sus4|maj6|min6|7|maj7|min7|dim7|hdim7|minmaj7|aug7|9|maj9|min9|11|maj11|min11|13|maj13|min13)(\\((\\*?((b*|#*)([1-9]|1[0-3]?))(,\\*?((b*|#*)([1-9]|1[0-3]?)))*)\\))?)|(:\\((\\*?((b*|#*)([1-9]|1[0-3]?))(,\\*?((b*|#*)([1-9]|1[0-3]?)))*)\\)))?((/((b*|#*)([1-9]|1[0-3]?)))?)?))$'

Failed validating 'pattern' in schema['items']['properties']['value']:
    {'type': 'string',
     'pattern': '^((N|X)|(([A-G](b*|#*))((:(maj|min|dim|aug|1|5|sus2|sus4|maj6|min6|7|maj7|min7|dim7|hdim7|minmaj7|aug7|9|maj9|min9|11|maj11|min11|13|maj13|min13)(\\((\\*?((b*|#*)([1-9]|1[0-3]?))(,\\*?((b*|#*)([1-9]|1[0-3]?)))*)\\))?)|(:\\((\\*?((b*|#*)([1-9]|1[0-3]?))(,\\*?((b*|#*)([1-9]|1[0-3]?)))*)\\)))?((/((b*|#*)([1-9]|1[0-3]?)))?)?))$'}

On instance[0]['value']:
    'ii (Dmin7)'